In [1]:
from MKLpy.preprocessing import normalization, rescale_01
from sklearn.metrics.pairwise import rbf_kernel as RBF
import os
import pandas as pd
import numpy as np
import pickle as pkl
from MKLpy.metrics.pairwise.misc import homogeneous_polynomial_kernel as HPK_kernel
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [37]:
extPath= '/media/ak/My Passport/Experiment Data'
featuresPath ="/".join((extPath,'features'))
labelsPath ="/".join((extPath,'labels'))
symbols = list(set(os.listdir(featuresPath))& set(os.listdir(labelsPath)))
MKLExpPath = "/".join((MKLPath,symbol,'Experiments' )) #symbol is in symbols
MKLSymbolPath = "/".join((MKLPath,symbol)) #symbol is in symbols
MKLKernPath = "/".join((MKLPath,symbol,'Kernels' )) #symbol is in symbols

In [44]:
def checkDir(dirLoc):
    print
    if os.path.exists(dirLoc):  # Checks if the dir exists
        print("The directory exists")
    else:
        print("No directory found for " + dirLoc)  # Output if no directory

In [7]:
MKLPath = "/".join((extPath,'MKLExpPath'))

path =MKLPath
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /media/ak/My Passport/Experiment Data/MKLExpPath 



try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

In [46]:
symbolIdx =2
symbol =symbols[symbolIdx]
symbolFeaturesLocation = "/".join((featuresPath,symbol,'MODEL_BASED'))
symbolLabelsLocation = "/".join((labelsPath,symbol,'NON_DIRECTIONAL')) #location of labels for each symbol
symbolHMMDatesList = os.listdir(symbolFeaturesLocation) 
# list of all the MODEL dates we have generated features files for. each #
# each of these dates in symbolFeaturesDates corresponds to a list of dates
# (symbolHMMDatesList = '20170829', '20170710', '20170801', ... ]
symbolLabelsDates = [dateFile.split(".")[0] for dateFile in os.listdir(symbolLabelsLocation)]

In [47]:
# now lets go down into each HMM-model date, and pick all the forward futures (out of sample)
hmmFeatureLocations = {}  # symbol-hmm-model-date index --> this is the indexation in symbolFeaturesDatesList
commonDatesDict = {}  # this is a struct that will contain for each HMM date, the common labels/features- this should
# be used for training and testing
createDate = []  # place holder for the hash key of when the features got created
symbolEachModelFeaturesDates = {}
HMMModelFeaturesLabelsCommon = {}  # location dictionary with 2 keys: HMM Date and Common Date
commonDates = []

LocDictsList = []

In [53]:
hmmDateIdx=1
symbolModelFeaturesDate = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx])


IndexError: list index out of range

In [52]:
for hmmDateIdx, hmmDate in enumerate(sorted(symbolHMMDatesList)):
    symbolModelFeaturesDate = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx])
    create_date = os.listdir(symbolModelFeaturesDate)[3].split("_")[-2]
    # output looks like this: /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170710
    symbolEachModelFeaturesDates[symbolHMMDatesList[hmmDateIdx]] = [file.split("_")[5] for file in
                                                                    os.listdir(symbolModelFeaturesDate)]
    # output is a dictionary where the keys are the HMM models dates and the values a list of dates - for each HMM
    # date we have a list of features
    for keyHMMDate in sorted(list(symbolEachModelFeaturesDates.keys())):  # for each of the HMM model dates
        print(keyHMMDate)
        common_dates = list(set(symbolEachModelFeaturesDates[keyHMMDate]) & set(symbolLabelsDates))
        # take the list of feature dates (conditional on HMM model date) + the list of labels -intersection!

        '''we now produce a dict for each HMM model, where each value is a list of common dates and we are key-ed by 
        the HMM Date '''
        commonDatesDict[keyHMMDate] = common_dates
        for commonDate in common_dates:
            '''iterate through all the common dates and figure out the location of each file for labels and 
            features '''
            labelsCommonFileLoc = "/".join((symbolLabelsLocation, ".".join((commonDate, 'csv'))))
            #             comnDateFeatureLocMaster = os.path.join((symbolModelFeaturesDate, commonDate))
            commonDatesFeatureFile = "".join(
                (symbols[symbolIdx], '_3_states_features_date:_', commonDate, "_now:_", create_date, "_.pickle"))
            FeatureFileLoc = os.path.join(symbolModelFeaturesDate, commonDatesFeatureFile)
            checkDir(FeatureFileLoc)
            checkDir(labelsCommonFileLoc)
            conditions = [os.path.exists(FeatureFileLoc), os.path.exists(labelsCommonFileLoc)]
            print(conditions)
            if all(conditions):
                print('all good on Date:', commonDate)
                HMMModelFeaturesLabelsCommon[keyHMMDate, commonDate] = [FeatureFileLoc, labelsCommonFileLoc]
                pkl.dump(HMMModelFeaturesLabelsCommon,
                         open("/".join((MKLSymbolPath, "LocDictsListCorrect.pkl")), "wb"))

            else:
                print('problem on date: ', commonDate)
                continue
            pkl.dump(commonDatesDict,
                     open("/".join((MKLSymbolPath, "CommonLocationsDicts.pkl")), "wb"))

    # LocDictsList.extend(commonDatesDict)
    # LocDictsList.extend(HMMModelFeaturesLabelsCommon)

# return LocDictsList
